In [11]:
import sqlite3
import pandas as pd
import os
import sys
from Bio.SubsMat import MatrixInfo as matlist
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Data import CodonTable
from Bio.Seq import translate
import math
import numpy as np
def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

conn = sqlite3.connect('../../mismatch_db.db')

In [45]:
mismatch = pd.read_sql_query("SELECT * FROM mismatch", conn)
mismatch = mismatch.astype({"exon_start_prim": "Int64", "exon_stop_prim":"Int64", "exon_start_hum": "Int64", "exon_stop_hum":"Int64"})

prim_exon_introns = pd.read_sql_query("""
SELECT mismatch_ID, mismatch.prot_prim, exon_intron_map.'type', exon_intron_map.number_elem, exon_intron_map.seq
FROM mismatch
JOIN protein ON mismatch.prot_prim = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID
JOIN exon_intron_map ON transcript.transcript_ID = exon_intron_map.transcript_ID
""", conn)

mismatch_prot_transcript = pd.read_sql_query(
"""SELECT * 
FROM mismatch
JOIN protein ON mismatch.prot_prim = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID""", conn)

mismatch_to_correct = pd.read_sql_query(
"""SELECT mismatch.*
FROM mismatch_flag 
JOIN tblastn_match ON mismatch_flag.mismatch_ID = tblastn_match.mismatch_ID 
JOIN mismatch on mismatch_flag.mismatch_ID = mismatch.mismatch_ID 
WHERE (one_hum_multiple_prim = 1
OR non_canonical_hum_spl = 1
OR N_in_genomic = 1
OR small_introns = 1)
AND conserved = 0
AND repeats_prot = 0
AND alignement_error = 0
AND human_isoform_exist = 0
""", conn)

In [19]:
prim_exon_introns

,mismatch_ID,prot_prim,type,number_elem,seq
0,0,A0A0D9RC82,Exon,1,CACCGCTGGCCAGACGAGGGATGTTATTTTGGGCAGTGCATCCGGA...
1,0,A0A0D9RC82,Exon,2,CATCTACAAATCTGAAAGACAAAACATGGTTCAAACATCCAGGCAC...
2,0,A0A0D9RC82,Exon,3,GTCCACCCGTGGCTCCAAAATGGTCTCCTGGTCCGTGACAGCAAAG...
3,0,A0A0D9RC82,Exon,4,GTGTTCGGCCTTGGTTCCGTGGCCCATATGGTTCTAAATAAAAAAT...
4,0,A0A0D9RC82,Exon,5,GGGCCCACATGAACGCAGCTGTGACCTTCGCTAACTGTGCACTGGG...
...,...,...,...,...,...
383254,11014,H0WR01,Intron,17,GTAAGTTTATATATTGACTAGGGCTTCATCTTTTATTTTGGAATCA...
383255,11014,H0WR01,Intron,18,GTGAGTGATGTGAGTTTACTTCCTTTTCTGAAGATGATGATGAACT...
383256,11014,H0WR01,Intron,19,GTAAAAAGCCAATACTATAAAGCTGTGTTCCCTTCCTGTCATTTTG...
383257,11014,H0WR01,Intron,20,GTAAATTTGTCATTTGAGGCATTTTCCAATACAAATATTGTTTATA...


In [435]:
# SCRIPT DE CORRECTION DES MISMATCH

for index, row in mismatch.iloc[12:13,:].iterrows():
    transcript_map = prim_exon_introns.loc[(prim_exon_introns["mismatch_ID"] == int(row[0])) & (prim_exon_introns["type"] == "Exon")]
    id_exon_start = row[7]-1
    id_exon_stop = row[8]
    mismatch_peptid = row[11]
    
    # LOCALISATION EXACT DES MISMATCH
    mismatch_exon_table = transcript_map.iloc[id_exon_start:id_exon_stop]
    mismatch_exon = transcript_map.iloc[id_exon_start:id_exon_stop, 4]
    exon_seq = mismatch_exon.str.cat(sep="")
    for i in range(3):
        peptid = translate(exon_seq[i:])
        frame = i
        match_find = peptid.find(row[11])
        if match_find != -1:
            break
    start_nuc_mismatch = match_find*3+frame
    stop_nuc_mismatch = match_find*3+len(mismatch_peptid*3)+frame
    
    # CREATION DES COORDONEE NUCLEOTIDES DES EXONS A MISMATCH
    taille_exon = []
    for index2, row2 in mismatch_exon_table.iloc[:,:].iterrows():
        taille_exon.append(len(row2[4]))
    taille_exon_fin = [sum(taille_exon[:x+1]) for x in range(0,len(taille_exon))]
    taille_exon_debut = taille_exon_fin.copy()
    taille_exon_debut.pop()
    taille_exon_debut[:]=[i+1 for i in taille_exon_debut]
    taille_exon_debut.insert(0, 0)
    mismatch_exon_table["pos_start"] = taille_exon_debut
    mismatch_exon_table["pos_stop"] = taille_exon_fin
    
    # SUPPRESSION DES PARTIE D'EXON DES TRANSCRIPT
    # Print Prot
    exon_seq = transcript_map.iloc[:, 4].str.cat(sep="")
    #print(transcript_map)
    print(translate(exon_seq[:]))
    #print("*")
    print(row[2])
    print(frame)
    print(row[11])
    #print("*")
    for index2, row2 in mismatch_exon_table.iloc[:,:].iterrows():
        if row2[5] >= start_nuc_mismatch and row2[6] <= stop_nuc_mismatch:
            # OK TOTALLY
            print("TO DELETE TOTALLY", row2[3])
            conditional_index = transcript_map.loc[ transcript_map['number_elem'] == row2[3] ].index
            transcript_map.loc[conditional_index, ["seq"]] = ""

            # FRAME OK +1
        if row2[5] < start_nuc_mismatch and row2[6] < stop_nuc_mismatch:
            print("DELETE THE END OF EXON", row2[3])
            conditional_index = transcript_map.loc[ transcript_map['number_elem'] == row2[3] ].index
            suppr = transcript_map.loc[conditional_index, "seq"].iloc[0][start_nuc_mismatch-row2[5]:]
            transcript_map.loc[conditional_index, ["seq"]] = transcript_map.loc[conditional_index, "seq"].iloc[0][:start_nuc_mismatch-row2[5]]
        if row2[5] > start_nuc_mismatch and row2[6] > stop_nuc_mismatch:
            print("DELETE THE BEGIN OF EXON", row2[3])
            conditional_index = transcript_map.loc[ transcript_map['number_elem'] == row2[3] ].index
            suppr += transcript_map.loc[conditional_index, "seq"].iloc[0][:stop_nuc_mismatch-row2[5]+1]
            transcript_map.loc[conditional_index, ["seq"]] = transcript_map.loc[conditional_index, "seq"].iloc[0][stop_nuc_mismatch-row2[5]+1:]

        if row2[5] <= start_nuc_mismatch and row2[6] >= stop_nuc_mismatch:
        # INSIDE 2 FRAMES OK
            print("DELETE INSIDE THE EXON", row2[3], " ", start_nuc_mismatch, " ", stop_nuc_mismatch)
            # 1er nvx exon 
            conditional_index = transcript_map.loc[ transcript_map['number_elem'] == row2[3] ].index
            transcript_map.loc[conditional_index, ["seq"]] = transcript_map.loc[conditional_index, "seq"].iloc[0][:start_nuc_mismatch-row2[5]]
            # 2e nvx exon exon X.5
            new_row = row2.copy()
            new_row[3] = new_row[3] + 0.5
            new_row[4] = new_row[4][stop_nuc_mismatch-new_row[5]:]
            row_df = pd.DataFrame(new_row)
            if len(row_df.index) > 1: row_df = row_df.T
            transcript_map = pd.concat([transcript_map, row_df], ignore_index=True)
            conditional_index = transcript_map.loc[ transcript_map['number_elem'] == row2[3]+0.5 ].index
            transcript_map.sort_values(by="number_elem", inplace=True, ignore_index=True)
            transcript_map["number_elem"] = range(1, len(transcript_map.index)+1)
    exon_seq = transcript_map.iloc[:, 4].str.cat(sep="")
    print(translate(exon_seq[:]))
    print("###############################################################")

# 1/ Retrouver le cadre contenant le mismatch  ; Trouver la meilleur CDS
# Final: mapper les exons et checker si les start s'enchainent

MPTLVMGMLPTCLGDTPQPGHKNSQRQGPFSHGAPGRAAGWEAVAKPRLRAPAAEDDVAALRWPGPSQQPDPPWAAPHVVGSEDLEEPGPWGKACSLPMWSTGPEARDGDSSVSSGRLSCSSGGHEACVSWKERPPQVLGPQWRPRKSDPRLEQLRDKIRAQVWRQESCASLGTSAPSRASRLHKASALAHRRKTQEAKNPPPAPECSGFSILSAAERRVEAKASHGQGRELSRVSQHQVPVRREKPKRVKSSSCRREKTPKSPSPRRVAKDKDEDSELVGVYAWRKGQALVRSLLGPPPVLRRQHSKDPSRDPALTVDSGDSEKVIAAECSPVCAQLPGATSAHSDQQVSGNTPSLASFNQPATIQTAMAILQDLRQQIQAGLELAQARNGGQELGPSKQRLQDVAGRGCCRDPSAQSSFSKGSCTMTERKHSSSERARSFHAWEPWSSSTGRESCPQRAWGAQGQDRSFQRPESPHKRLGHFSQRPWSALAGQACSPQRASGAQRQGPSSQRPDSPPEKRSPFPQQPWSAAATQPCPRRAWTACEDWEAPGPRLWNPLERPSPPAQRPWSSSGVQRAGPQGKGRGLGPPASAAKHVLPRPTRSFPQNPLGKEKDALRPCPRPRGLLGPSHSSESLREFIRQKARARRRQALEEKASALRTRELRSRRLQEVYRQQREAILGRAVPVVSRTTPGIVTFVPSSAQSGVPMGPRLGRSGAASSPLVSSREWPGEGCSWGLGTSWCHLCRWVSVPCWPRCVTPSPRAAPPPLATVGLRLLDSLPSSWDLSSRAAPGTVAPPCNPSTLGGQEFETGLDNIVRPWSEARCPLPQGSPEEGHMDVKLDKRLQRGVAPFQALSPSSGSSHAGPATLHPIWGSLRLEETPSAGGADSVAPWSPRSCGKGDPVDRPRAGWSGGRGSQGTHWGDCCRLCPTRPVPSSRTLTFSWVTAQSSGTNSSCPRTSPDLLVADMAGDEQDSPSSALGALGRFKLQMLERSLREEE

/home/meyer/anaconda3/envs/stage-env/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/meyer/anaconda3/envs/stage-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [368]:
print(len(transcript_map.loc[conditional_index, "seq"].iloc[0][start_nuc_mismatch-row2[5]:]))

15


In [318]:
transcript_map

,mismatch_ID,prot_prim,type,number_elem,seq,pos_start,pos_stop
0,1,H2P532,Exon,1,CACCTTACTCTGCACCAGAGAATTCATACTGGGGAGAAACCCTATG...,NaN,NaN
1,0,A0A0D9RC82,Exon,8.5,GTGGCATCATCTACCTGGTCTTCATTGGCTCCACCATCCCACGGGA...,219,2936


In [283]:
transcript_map

,mismatch_ID,prot_prim,type,number_elem,seq
0,0,A0A0D9RC82,Exon,1,CACCGCTGGCCAGACGAGGGATGTTATTTTGGGCAGTGCATCCGGA...
1,0,A0A0D9RC82,Exon,2,CATCTACAAATCTGAAAGACAAAACATGGTTCAAACATCCAGGCAC...
2,0,A0A0D9RC82,Exon,3,GTCCACCCGTGGCTCCAAAATGGTCTCCTGGTCCGTGACAGCAAAG...
3,0,A0A0D9RC82,Exon,4,GTGTTCGGCCTTGGTTCCGTGGCCCATATGGTTCTAAATAAAAAAT...
4,0,A0A0D9RC82,Exon,5,GGGCCCACATGAACGCAGCTGTGACCTTCGCTAACTGTGCACTGGG...
5,0,A0A0D9RC82,Exon,6,CGGCCATTCTCCACTTTTCGGGTGGACAGCTGATGGTGACCGGTCC...
6,0,A0A0D9RC82,Exon,7,GCGTGGCTGACCGGGATGCTCCAGCTGTGTCTCTTCGCCATCACGG...
7,0,A0A0D9RC82,Exon,8,ATGAATCCATGGCCCTAGAGCACTTCTAAGTAGAGATTATTTGTGA...


In [172]:
# SUPPRESSION DES PARTIE D'EXON DES TRANSCRIPT
for index2, row2 in mismatch_exon_table.iloc[:,:].iterrows():
    if row2[5] >= start_nuc_mismatch and row2[6] <= stop_nuc_mismatch:
        print("TO DELETE TOTALLY")
    
    if row2[5] <= start_nuc_mismatch and row2[6] <= stop_nuc_mismatch:
        print("DELETE THE END OF EXON")
        
    if row2[5] >= start_nuc_mismatch and row2[6] >= stop_nuc_mismatch:
        print("DELETE THE BEGIN OF EXON")

    if row2[5] <= start_nuc_mismatch and row2[6] >= stop_nuc_mismatch:
        print("DELETE INSIDE THE EXON")

DELETE THE END OF EXON
DELETE THE BEGIN OF EXON
